In [117]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
#import mysql_conn


In [118]:
tree_data = pd.read_csv('tree_data.csv')
tree_data.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03T00:00:00,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05T00:00:00,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05T00:00:00,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30T00:00:00,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [119]:
#obtain desired columns
tree_data = tree_data[['postcode','spc_common','spc_latin','tree_dbh']]
#clean data
tree_data_drop = tree_data.dropna()
tree_data_clean = tree_data_drop.drop_duplicates()

tree_data_clean.head()

,postcode,spc_common,spc_latin,tree_dbh
0,11375,red maple,Acer rubrum,3
1,11357,pin oak,Quercus palustris,21
2,11211,honeylocust,Gleditsia triacanthos var. inermis,3
3,11211,honeylocust,Gleditsia triacanthos var. inermis,10
4,11215,American linden,Tilia americana,21


In [120]:
#obtain average tree diameter
tree_info = tree_data_clean.groupby(['postcode','spc_common'])
tree_diam = pd.DataFrame(tree_info['tree_dbh'].mean())

#obtain number of tree type in postcode

tree_quantity = pd.DataFrame(tree_data_clean.groupby(['postcode','spc_common']).size())

tree_summary = tree_quantity.join(tree_diam).reset_index()
tree_summary = tree_summary.rename(columns={0:'tree_count'})
tree_summary.head(2)

,postcode,spc_common,tree_count,tree_dbh
0,83,American beech,2,7.500000
1,83,American elm,44,23.522727


In [121]:
engine = create_engine("mysql://root:<enterpassword>@localhost/tree_db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)


In [122]:
#rename columns to match database table
tree_transformed = tree_summary.rename(columns = {'postcode':'zip_code',
                              'spc_common': 'species_nm',
                              'tree_count': 'count_tree',
                              'tree_dbh': 'avg_diameter'})

In [123]:
#connect to database
engine = create_engine("mysql://root:<enter password>@localhost/tree_db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

OperationalError: (_mysql_exceptions.OperationalError) (2006, "Access denied for user 'root'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
#load dataframe to database
tree_transformed.to_sql(name='nyc_tree',con=engine,if_exists='append',index=False)

In [151]:
test_df = tree_data_clean['spc_common']
test_df_unique = pd.DataFrame(test_df.drop_duplicates())
#test_df_unique.str.strip()

test_df_unique['spc_common'] = test_df_unique['spc_common'].str.replace(" ","")
tree_df_new = test_df_unique.rename(columns={'spc_common':'web_value'})
tree_df_new['species_nm'] = tree_data_clean['spc_common']
tree_df_new

,web_value,species_nm
0,redmaple,red maple
1,pinoak,pin oak
2,honeylocust,honeylocust
4,Americanlinden,American linden
9,Londonplanetree,London planetree
16,ginkgo,ginkgo
22,willowoak,willow oak
23,sycamoremaple,sycamore maple
25,Amurmaple,Amur maple
65,hedgemaple,hedge maple
